# importing necessary libraries

In [39]:
import os
import argparse
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

# LangChain / OpenAI imports

In [41]:
pip install langchain openai faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [43]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document


# Helper functions

In [45]:
def load_csv(path, max_rows=20000):
    """Load CSV with a cap to avoid OOM on very large files."""
    df = pd.read_csv(path)
    if len(df) > max_rows:
        print(f"Warning: CSV has {len(df)} rows — truncating to first {max_rows} rows.")
        df = df.head(max_rows)
    return df


def row_to_text(row: pd.Series, id_col=None, max_chars=1000) -> str:
    """Convert a DataFrame row to a short text summary for embedding/retrieval."""
    parts = []
    if id_col and id_col in row.index:
        parts.append(f"id: {row[id_col]}")
    for c, v in row.items():
        parts.append(f"{c}: {v}")
    text = " | ".join(parts)
    if len(text) > max_chars:
        text = text[: max_chars - 3] + "..."
    return text


def build_docs_from_df(df: pd.DataFrame, id_col=None):
    docs = []
    for idx, row in df.iterrows():
        text = row_to_text(row, id_col=id_col)
        docs.append(Document(page_content=text, metadata={"row_index": int(idx)}))
    return docs

# Main agent builder

In [47]:
def create_vectorstore_from_csv(csv_path, id_col=None, openai_api_key=None):
    df = load_csv(csv_path)
    docs = build_docs_from_df(df, id_col=id_col)

    # Setup embeddings
    if openai_api_key is None:
        openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        raise ValueError("OpenAI API key not provided. Set OPENAI_API_KEY in environment or .env file.")

    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

    # Build FAISS in-memory index
    print("Creating vectorstore and embedding rows — this may take a few moments...")
    vs = FAISS.from_documents(documents=docs, embedding=embeddings)
    print(f"Indexed {len(docs)} rows into FAISS vectorstore.")
    return vs, df


def create_conversational_chain(vs, openai_api_key=None, temperature=0.2):
    if openai_api_key is None:
        openai_api_key = os.getenv("OPENAI_API_KEY")

    llm = ChatOpenAI(temperature=temperature, openai_api_key=openai_api_key)
    retriever = vs.as_retriever(search_type="similarity", search_kwargs={"k": 6})
    chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever)
    return chain

# Utilities for quick table summaries

In [49]:
def quick_table_summary(df: pd.DataFrame, max_cols=10) -> str:
    """Return a short textual summary of the table for the LLM context."""
    lines = []
    lines.append(f"Rows: {len(df)}")
    lines.append("Columns:")
    cols = df.columns.tolist()[:max_cols]
    for c in cols:
        lines.append(f" - {c} (dtype={df[c].dtype})")
    if len(df.columns) > max_cols:
        lines.append(f" ... and {len(df.columns)-max_cols} more columns")

    # Add basic statistics for numeric columns
    num = df.select_dtypes(include=["number"]).describe().T
    if not num.empty:
        lines.append("Numeric column summary (mean, std, min, max):")
        for c in num.index[:5]:  # limit to first 5 numeric cols
            r = num.loc[c]
            lines.append(f" - {c}: mean={r['mean']:.3f}, std={r['std']:.3f}, min={r['min']}, max={r['max']}")
    return "\n".join(lines)

# CLI / REPL

In [51]:
def repl(chain, df, history_limit=10):
    print("\n=== AI Data Agent REPL ===\nType 'exit' to quit, 'summary' for table summary, or ask questions about the data.\n")
    chat_history = []
    while True:
        q = input("You: ")
        if not q:
            continue
        q_stripped = q.strip().lower()
        if q_stripped in ("exit", "quit"):
            print("Goodbye — session ended.")
            break
        if q_stripped == "summary":
            print(quick_table_summary(df))
            continue

        # Ask the chain — it expects a dict with 'question' and optionally 'chat_history'
        res = chain.run({"question": q, "chat_history": chat_history})
        print("Agent:", res)

        # Update history (simple append)
        chat_history.append((q, res))
        if len(chat_history) > history_limit:
            chat_history = chat_history[-history_limit:]

# Main

In [53]:
def main():
    parser = argparse.ArgumentParser(description="Run a small AI agent over a CSV file.")
    parser.add_argument("--csv", required=True, help="Path to CSV file")
    parser.add_argument("--id_col", required=False, help="Optional column name to treat as ID")
    parser.add_argument("--no_embed", action="store_true", help="Skip building embeddings (not recommended) — just show summary")
    args = parser.parse_args()

    load_dotenv()  # load .env if present

    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("Warning: OPENAI_API_KEY not found in environment. Make sure to set it before running.")

    if args.no_embed:
        df = load_csv(args.csv)
        print(quick_table_summary(df))
        return

    vs, df = create_vectorstore_from_csv(args.csv, id_col=args.id_col, openai_api_key=api_key)
    chain = create_conversational_chain(vs, openai_api_key=api_key)
    repl(chain, df)


if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] --csv CSV [--id_col ID_COL] [--no_embed]
ipykernel_launcher.py: error: the following arguments are required: --csv


SystemExit: 2

In [67]:
pip install tiktoken

   ---------------------------------------- 0.0/875.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/875.7 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/875.7 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 875.7/875.7 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [71]:
df = pd.read_csv('marketing_campaign.csv')

In [ ]:
docs = []
for idx, row in df.iterrows():
    text = " | ".join([f"{col}: {val}" for col, val in row.items()])
    docs.append(Document(page_content=text, metadata={"row_index": idx}))

# Create embeddings
embeddings = OpenAIEmbeddings(openai_api_key="api_key_here")

# Create vectorstore
vs = FAISS.from_documents(docs, embeddings)

# Create conversational chain
chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0, openai_api_key="api_key_here"),
    retriever=vs.as_retriever()
)

In [ ]:
chat_history = []
question = "Which marketing channel had the highest ROI?"
answer = chain.run({"question": question, "chat_history": chat_history})
print(answer)